In [1]:
import pandas as pd
import os

def convert_data(file_path, output_dir):
    data = pd.read_csv(file_path, delimiter=',')
    data = data.iloc[:, :6]

    # 컬럼 이름 지정
    data.columns = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ']

    # 가속도 데이터 변환
    adxl345_range_value = 16  # ±16g
    adxl345_resolution = 13  # 13 bits
    adxl345_conversion_factor = (2 * adxl345_range_value) / (2 ** adxl345_resolution) 

    # 스마트워치 가속도 범위
    watch_acc_max_range = 78.4532

    # ADXL345 데이터를 변환하고, 스마트워치 범위로 스케일링
    data['accX'] = (data['accX'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)
    data['accY'] = (data['accY'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)
    data['accZ'] = (data['accZ'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)

    # 각속도 데이터 변환
    itg3200_range_value = 2000  # ±2000 °/s
    itg3200_resolution = 16  # 16 bits
    itg3200_conversion_factor = (2 * itg3200_range_value) / (2 ** itg3200_resolution) 

    # 스마트워치 자이로스코프 범위
    watch_gyro_max_range = 17.453642

    # 스케일링
    data['gyroX'] = (data['gyroX'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)
    data['gyroY'] = (data['gyroY'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)
    data['gyroZ'] = (data['gyroZ'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)

    output_file_path = os.path.join(output_dir, os.path.basename(file_path))
    data.to_csv(output_file_path, index=False)

In [2]:
def process_directory(input_dir, output_base_dir, label):
    output_dir = os.path.join(output_base_dir, label)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                convert_data(file_path, output_dir)

In [3]:
directories = ['./D01', './D05', './D12', './D14', './F01', './F02', './F08', './F09', './F13', './F15']
output_base_dir = './training_data'

# 각 디렉토리 처리
for directory in directories:
    label = 'false' if os.path.basename(directory).startswith('D') else 'true'
    process_directory(directory, output_base_dir, label)